In [1]:
import hashlib, math, os, subprocess
from multiprocessing import Process
import xlearn as xl
import numpy as np
import pandas as pd
from pandas import DataFrame as DF

In [2]:
data2=pd.read_csv("/data/ng/FFM/mid_tolist.csv")

In [3]:
data1=pd.read_csv("/data/data/final_data.csv")

In [7]:
data2.head()

,mid,timeMode,mid_list
0,1306505,3,"[0.052603, -0.040752, -0.056807, 0.020781, 0.1..."
1,10574468,8,"[-0.0025723305646388034, 0.004975539575848075,..."
2,4910186,6,"[-0.065039, 0.064973, 0.177405, 0.169921000000..."
3,1304643,3,"[-0.052975, -0.07658200000000001, -0.099915, 0..."
4,1851857,3,"[-0.126394, 0.047633, -0.15279700000000002, -0..."


In [7]:
data2.drop(labels=['name','director','scriptwriter','mainPlayers','categories','productCountry','duration','releasedTime','tags','ratingCount','star5','star4','star3','star2','star1','language'], axis=1,inplace = True)

In [12]:
data1.head()

,uid,mid,star,timeStamp
0,1722994,1306505,1,2
1,1405477,10574468,1,8
2,15849871,4910186,1,6
3,1068524,1304643,1,1
4,1307041,1851857,1,3


In [11]:
data1=data1.dropna()
data1['timeStamp']=data1['timeStamp'].apply(lambda x:int(x)-2005)

In [13]:
data3=pd.merge(data1,data2,on='mid')
data3=data3.dropna()

In [19]:
data3.head()

,uid,mid,star,timeStamp,timeMode,mid_list
0,1722994,1306505,1,2,3,"[0.052603, -0.040752, -0.056807, 0.020781, 0.1..."
1,1299639,1306505,1,4,3,"[0.052603, -0.040752, -0.056807, 0.020781, 0.1..."
2,1279487,1306505,1,3,3,"[0.052603, -0.040752, -0.056807, 0.020781, 0.1..."
3,1226283,1306505,1,1,3,"[0.052603, -0.040752, -0.056807, 0.020781, 0.1..."
4,1694045,1306505,1,3,3,"[0.052603, -0.040752, -0.056807, 0.020781, 0.1..."


In [13]:
def df_tolist(x):
    start=4
    end=54
    l=end-start
    temp=[0]*(l)
    for i in range(0,l):
        temp[i]=x[i+start]
    return temp

In [3]:
class FfmEncoder():
    def __init__(self, field_names, label_name, nthread=1):
        self.field_names = field_names
        self.nthread = nthread
        self.label = label_name

    def gen_feats(self, row):
        feats = []
        for field in self.field_names:
            value = row[field]
#             key = field + '-' + str(value)
            key=str(value)
            feats.append(key)
        return feats

    def gen_fm_feats(self, feats):
        feats = ['{0}:{1}:1'.format(field, feat) for (field, feat) in feats]
        return feats

    def convert(self, df, path, i):
        lines_per_thread = math.ceil(float(df.shape[0]) / self.nthread)
        sub_df = df.iloc[i * lines_per_thread: (i + 1) * lines_per_thread]
        tmp_path = path + '_tmp_{0}'.format(i)
        with open(tmp_path, 'w') as f:
            for index,row in sub_df.iterrows():
                feats = []
                for i, feat in enumerate(self.gen_feats(row)):
                    feats.append((i, feat))
                feats = self.gen_fm_feats(feats)
                f.write(str(int(row[self.label])) + ' ' + ' '.join(feats) + '\n')

    def parallel_convert(self, df, path):
        processes = []
        for i in range(self.nthread):
            p = Process(target=self.convert, args=(df, path, i))
            p.start()
            processes.append(p)
        for p in processes:
            p.join()

    def delete(self, path):
        for i in range(self.nthread):
            os.remove(path + '_tmp_{0}'.format(i))

    def cat(self, path):
        if os.path.exists(path):
            os.remove(path)
        for i in range(self.nthread):
            cmd = 'cat {svm}_tmp_{idx} >> {svm}'.format(svm=path, idx=i)
            p = subprocess.Popen(cmd, shell=True)
            p.communicate()

    def transform(self, df, path):
        print('converting data......')
        self.parallel_convert(df, path)
        self.cat(path)
        self.delete(path)

In [20]:
field_names = [i for i in data3.columns if i not in ['star']]

In [3]:
field_names =['uid','mid','timeStamp','timeMode','mid_list']

In [21]:
write_path = '/data/ng/FFM/'
fe = FfmEncoder(field_names,label_name='star',nthread=8)
fe.transform(data3, write_path+'train_without_last_mid.ffm')

converting data......


In [4]:
ffm_model = xl.create_ffm()
# ffm_model.setTrain("criteo.tr.r100.gbdt0.ffm")
ffm_model.setTrain("/data/ng/FFM/train_without_last_mid.ffm")
# ffm_model.setValidate("criteo.va.r100.gbdt0.ffm")
# ffm_model.setValidate("val.ffm")

param = {'task':'binary', # ‘binary’ for classification, ‘reg’ for Regression
         'k':2,           # Size of latent factor
         'lr':0.1,        # Learning rate for GD
         'lambda':0.0002, # L2 Regularization Parameter
#          'Metric':'auc',  # Metric for monitoring validation set performance
         'epoch':25       # Maximum number of Epochs
        }

ffm_model.fit(param, "/data/ng/model.out")

In [ ]:
ffm_model.setTest("val.ffm")
ffm_model.setSigmoid()
ffm_model.predict("/data/ng/model.out", "output1.txt")

In [2]:
ffm_model = xl.create_ffm()